In [1]:
import os
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
from tqdm import tqdm
import time
from PIL import Image

from skew_ml import SkewDataset, ResNetRegression, train_model, transform

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train_dataset = SkewDataset(csv_file='/kaggle/input/deskew/label/train.csv',
                            root_dir='/kaggle/input/deskew/images/train',
                            transform=transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

valid_dataset = SkewDataset(csv_file='/kaggle/input/deskew/label/valid.csv',
                            root_dir='/kaggle/input/deskew/images/valid',
                            transform=transform)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)

In [ ]:
modelname="resnet50"

In [ ]:
model = ResNetRegression(modelname).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
train_model(model, criterion, optimizer, train_loader, valid_loader, num_epochs=50, num_save=2)

In [ ]:
# resnet18
# !curl -O -L "https://huggingface.co/sree711/resnetregression/resolve/main/regressionnet18.pth?download=true"

In [ ]:
# resnet50
# !curl -O -L "https://huggingface.co/sree711/resnetregression/resolve/main/regressionnet50.pth?download=true"

In [ ]:
# checkpoint = torch.load("regressionnet50.pth")
# model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
checkpoint = torch.load("/kaggle/input/resnet50-regression/chkmodel35.pth")
model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
csv_file =  "/kaggle/input/deskew/label/test.csv"
root_dir = "/kaggle/input/deskew/images/test"

In [ ]:
test_dataset = SkewDataset(csv_file=csv_file,
                                root_dir=root_dir,
                                transform=transform)

test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)
model.eval()
total_loss = 0.0
criterion = torch.nn.L1Loss()
with torch.no_grad():
    mean_preprocess = 0
    mean_inference = 0
        
    for images, angles in tqdm(test_loader, unit='it'):
        start_preprocess = time.time()
        images = images.to(device)
        angles = angles.to(device)
        end_preprocess = time.time()
        mean_preprocess += (end_preprocess - start_preprocess) * 1000
            
        start_inference = time.time()
        outputs = model(images)
        end_inference = time.time()
        mean_inference += (end_inference - start_inference) * 1000  
            
        loss = criterion(outputs, angles.float())
        total_loss += loss.item() * images.size(0)

mean_preprocess /= len(test_loader)
mean_inference /= len(test_loader)

mean_loss = total_loss / len(test_loader.dataset)
print(f"Mean Preprocessing Time: {mean_preprocess:.4f} ms")
print(f"Mean Inference Time: {mean_inference:.4f} ms")
print(f"Test Loss: {mean_loss:.4f}")

In [ ]:
def predict_skew_angle(model, image_path):
    image = Image.open(image_path)
    input_image = transform(image).unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        output = model(input_image)
        
    predicted_angle = output.item()
    print(f"Predicted skew angle: {predicted_angle:.3f} degrees")

In [ ]:
file = os.listdir("/kaggle/input/deskew/images/test")[1]
predict_skew_angle(model, os.path.join("/kaggle/input/deskew/images/test", file))

In [ ]:
import pandas as pd
csv_path = '/kaggle/input/deskew/label/test.csv'
df = pd.read_csv(csv_path)
row = df[df['filename'] == os.path.basename(file)]
if not row.empty:
    print(row)